In [ ]:
import gzip
import os
import re
import sys
from urllib.request import urlretrieve
from getpass import getpass

import pandas as pd
from mysql.connector import Error, connect, errorcode

## Prepare: Setting parameters

In [ ]:
# Configurate MySQL connection

# input username and password when run this block
CONFIG = {
    "host": "localhost",
    "user": input("Username: "),
    "password": getpass("Password: "),
}

# MySQL database name
DB_NAME = "my_imdb"

# Datasets folder location
DATASET_LOC = os.path.abspath("datasets")

# List of datasets download links
URLS = [
    "https://datasets.imdbws.com/title.basics.tsv.gz",
    "https://datasets.imdbws.com/name.basics.tsv.gz",
    "https://datasets.imdbws.com/title.ratings.tsv.gz",
    "https://datasets.imdbws.com/title.crew.tsv.gz"
    ]

# Whether overwrite existing datasets
OVERWRITE = input("Overwrite existing files? (y/n) ").lower()
if OVERWRITE == "y":
    OVERWRITE = True
elif OVERWRITE == "n":
    OVERWRITE = False
else:
    sys.exit("Invalid input, please try again.")

# Queries to create tables
CREATES = {}
CREATES["title_basics"] = """
    CREATE TABLE title_basics (
        tconst varchar(20) NOT NULL,
        titleType varchar(20),
        primaryTitle varchar(1000),
        originalTItle varchar(1000),
        isAdult bool,
        startYear smallint,
        endYear smallint,
        runtimeMinutes int,
        genres varchar(255),
        PRIMARY KEY (tconst)
    )
    """

CREATES["name_basics"] = """
    CREATE TABLE name_basics (
        nconst varchar(20) NOT NULL,
        primaryName varchar(50),
        birthYear smallint,
        deathYear smallint,
        primaryProfession varchar(100),
        knownForTitles varchar(100),
        PRIMARY KEY (nconst),
        FOREIGN KEY (knownForTitles)
            REFERENCES title_basics(tconst)
    )
    """

CREATES["title_ratings"] = """
    CREATE TABLE title_ratings (
        tconst varchar(20) NOT NULL,
        averageRating decimal(3, 1),
        numVotes int,
        PRIMARY KEY (tconst),
        FOREIGN KEY (tconst)
            REFERENCES title_basics(tconst)
    )
    """

CREATES["title_crew"] = """
    CREATE TABLE title_crew (
        tconst varchar(20) NOT NULL,
        directors varchar(20),
        writers varchar(20),
        PRIMARY KEY (tconst),
        FOREIGN KEY (tconst)
            REFERENCES title_basics(tconst),
        FOREIGN KEY (directors)
            REFERENCES name_basics(nconst),
        FOREIGN KEY (writers)
            REFERENCES name_basics(nconst)
    )
    """


# Queries of inserting records
INSERTS = {}
INSERTS["title_basics"] = """
    INSERT INTO title_basics
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

INSERTS["name_basics"] = """
    INSERT INTO name_basics
    VALUES (%s, %s, %s, %s, %s, %s)
    """

INSERTS["title_ratings"] = """
    INSERT INTO title_ratings
    VALUES (%s, %s, %s)
    """

INSERTS["title_crew"] = """
    INSERT INTO title_crew
    VALUES (%s, %s, %s)
    """



In [ ]:
if not os.path.exists(DATASET_LOC):
    os.makedirs(DATASET_LOC)

## Part 1: Donwload and Extract Data

In [ ]:
# Function to prompt skip overwriting existing file
def check_overwrite(filename):
    if os.path.exists(os.path.join(DATASET_LOC, filename)):
        return OVERWRITE
    else:
        return


In [ ]:
# Create a Filename class to conatins attributes of different filenames
class Filename:
    def __init__(self, name, url, zip, tsv, csv, small):
        self.name = name
        self.url = url
        self.zip = zip
        self.tsv = tsv 
        self.csv = csv
        self.small = small

In [ ]:
# Get a dictionary of filenames from the donwload link
filenames = {}
for url in URLS:
    zip = url.rsplit("/", 1)[-1]
    tsv_matches = re.search(r"^(.+)\.(.+)\.(tsv).gz$", zip)
    name = f"{tsv_matches[1]}_{tsv_matches[2]}"
    tsv = os.path.join(name + os.extsep + "tsv")
    csv = os.path.join(name + os.extsep + "csv")
    small = os.path.join(name + "_s" + os.extsep + "csv")
    filenames[name] = Filename(name, url, zip, tsv, csv, small)

In [ ]:
# Donwload datasets files from imdb website
for filename in filenames.values():
    if check_overwrite(filename.zip) == False:
        continue
    try:
        urlretrieve(filename.url, os.path.join(DATASET_LOC, filename.zip))
        print(f"Downloaded '{filename.zip}' to {DATASET_LOC}")
    except Error as e:
        print(f"Download '{filename.zip}' fail")


In [ ]:
# Unzip and rename the tsv files
for filename in filenames.values():
    if check_overwrite(filename.tsv) == False:
        continue
    try:
        with gzip.open(os.path.join(DATASET_LOC, filename.zip), "rb") as f_in:
            with open(os.path.join(DATASET_LOC, filename.tsv), "wb") as f_out:
                f_out.write(f_in.read())
                print(f"Extracted {filename.tsv} to {DATASET_LOC}")
    except Error as e:
        print(f'Extract {filename.zip} Failed')

In [ ]:
# Data cleaning and convert tsv to csv file
for filename in filenames.values():
    imdb_table = pd.read_table(os.path.join(DATASET_LOC, filename.tsv), sep="\t")
    
    # Data Cleaning for title_basics
    if filename.name == "title_basics":

        # Locate all rows with primaryTilte issues
        title_issue_df = imdb_table[imdb_table["primaryTitle"].str.contains(r".+\t.+") == True]
        
        # If rows with primaryTilte issues exit
        if title_issue_df.shape[0] > 0:
            rows_fixed = 0
            for index, row in title_issue_df.iterrows():
                values = row.values.flatten().tolist()
                
                # Split the string to two columns
                clean_titles = values[2].split("\t")
                values[2] = clean_titles[0]
                values.insert(3, clean_titles[1])

                # Removed unnecessary NaN value at the end
                values.pop()

                # Replace the row in the table
                imdb_table[imdb_table["tconst"] == values[0]] = values
                rows_fixed += 1
            print(f"Fixed {rows_fixed} row")
        
    # Export csv
    if check_overwrite(filename.csv) == False:
        continue
    try:
        imdb_table.to_csv(os.path.join(DATASET_LOC, filename.csv), index=False)
        print(f"Converted {filename.tsv} to {filename.csv}")
    except:
        print(f"Convert {filename.tsv} failed")

In [ ]:
# Optional
# Reduce rows in files and convert to csv
for filename in filenames.values():
    if filename.name == "name_basics":
        continue
    else:
        imdb_table = pd.read_table(os.path.join(DATASET_LOC, filename.tsv), sep="\t")
        imdb_table_s = imdb_table[imdb_table["tconst"] <= "tt0000100"]
        filename.csv = filename.small
    if check_overwrite(filename.small) == False:
        continue
    try:
        imdb_table_s.to_csv(os.path.join(DATASET_LOC, filename.small), index=False)
        print(f"Converted {filename.tsv} to {filename.small}")
    except:
        print(f"Convert {filename.tsv} failed")


In [ ]:
# Read csv and more data cleaning
imdb_df = {}
for filename in filenames.values():
    imdb_df[filename.name] = pd.read_csv(os.path.join(DATASET_LOC, filename.csv), index_col=False)
    temp_df = imdb_df[filename.name]
    replace = 0
    
    # Replace \N with None
    temp_df.replace(r"\\N", None, regex=True, inplace=True)
    replace += 1

    # Replace NaN with None
    imdb_df[filename.name] = temp_df.where(pd.notnull(temp_df), None)
    replace += 1
    print(f"Made {replace} replacements on {filename.name}")


## Part 2: Inserting Data to MySQL
\* Start MySQL Server before running this part

In [ ]:
# Connect/Create database
cnx = connect(**CONFIG)
cursor = cnx.cursor()

try : 
    # Connect to database
    cursor.execute(f"USE {DB_NAME}")
    print(f"Connected to {DB_NAME} database")
except Error as e:
    if e.errno == errorcode.ER_BAD_DB_ERROR:
        print(f"Database {DB_NAME} does not exists.")
        
        # Create database if not alrady exist
        cursor.execute(f"CREATE DATABASE {DB_NAME}")
        print(f"{DB_NAME} database created.")
        cnx.database = DB_NAME
    else:
        print(f"Failed to connect to {DB_NAME} database")


In [ ]:
# Create tables
for table in CREATES:
    create_query = CREATES[table]
    try:
        cursor.execute(create_query)
        print(f"Created table {table}")
    except Error as e:
        if e.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print(f"{table} already exist")
        else:
            print(f"Fail to create {table}: {e}")


In [ ]:
# Insert_row(cursor, table, df):
for table in INSERTS:
    insert_query = INSERTS[table]
    insert_rows = 0
    try:
        for index, row in imdb_df[table].iterrows():
            insert_data = tuple(row)
            cursor.execute(insert_query, insert_data)
            cnx.commit()
            insert_rows += 1
    except Error as e:
        if e.errno == errorcode.ER_DUP_ENTRY:
            pass
        else:
            print(f"Insert failed at row {index}: {insert_data}")
            print(e)
    print(f"Inserted {insert_rows} rows to {table}")

In [ ]:
# Closed MySQL connection
# Always run this block when finished the program
cursor.close
cnx.close

## Part 3 Quering the Database

In [ ]:
query = "SELECT * FROM title_ratings WHERE averageRating > 3 LIMIT 5"
query1 = """
    SELECT title_basics.tconst, title_basics.primaryTitle, title_basics.startYear, CAST(title_ratings.averageRating as FLOAT)
    FROM title_basics
    JOIN title_ratings
        ON title_basics.tconst = title_ratings.tconst
    WHERE title_basics.startYear = 1896
    ORDER BY title_ratings.averageRating DESC
    LIMIT 10
    """

In [ ]:
try: 
    with connect(**CONFIG) as cnx:        
        with cnx.cursor() as cursor:
            cnx.database = DB_NAME
            cursor.execute(query1)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)